In [1]:
from pathlib import Path
from common.data.amigos.utils import load_participant_data
from common.data.amigos.config import AmigosConfig
import mne.io

from common.data.eeg.saliency_extractor import EEGFeatureExtractor

info = mne.create_info(
    ch_names=AmigosConfig.CH_NAMES,
    ch_types=AmigosConfig.CH_TYPES,
    sfreq=AmigosConfig.original_eeg_fs
)
participant_data = load_participant_data(Path("../../../resources/AMIGOS/pre_processed_py/"))
eeg_data = participant_data["P01"]["joined_data"][0]
raw = mne.io.RawArray(eeg_data.T, info=info, verbose=False)

In [2]:
extractor = EEGFeatureExtractor(raw)
starts = extractor.pick_segments(
    4, 0.125, bands=((4, 8), (8, 13), (13, 30)), band_weights=(0.4, 0.5, 0.4)
)

Finding segments below or above PTP threshold.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 57 samples (0.445 s)

Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 213 sam

In [3]:
from common.data.sampler import FeatureAndRandomLogUniformIntervalsSegmenter

feat_seg = FeatureAndRandomLogUniformIntervalsSegmenter(
    min_length=1,
    max_length=30,
    num_segments=20,
    extraction_jitter=0,
    anchor_identification_hop=0.125
)

In [7]:
from common.data.eeg import EEG
import numpy as np

r = feat_seg.compute_segments(EEG(data=raw, eid="1", fs=128))
a = np.array(r)

np.diff(a)

Finding segments below or above PTP threshold.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 57 samples (0.445 s)

Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 213 sam

array([[ 4.       ],
       [ 1.1484375],
       [ 1.1484375],
       [ 1.390625 ],
       [ 4.       ],
       [ 4.       ],
       [ 4.       ],
       [ 7.2890625],
       [ 7.6875   ],
       [ 8.4921875],
       [ 8.6171875],
       [ 9.8671875],
       [12.03125  ],
       [16.8359375],
       [17.546875 ],
       [17.8828125],
       [19.6171875],
       [22.5546875]])

In [ ]:
a

In [8]:
# Total coverage:
unique_coverage = 0
duration = raw.duration

taken_quarter_seconds = np.array([0 for i in range(int(duration * 4))])
for start, stop in a:
    start = int(start * 4)
    stop = int(stop * 4)
    taken_quarter_seconds[start:stop] += 1

unique_coverage = len(taken_quarter_seconds[taken_quarter_seconds > 0]) / (raw.duration * 4)
print(unique_coverage)  # We use 95% of the original signal

0.95280163599182


In [ ]:
# Coverage of SHORT segments:
unique_coverage = 0
duration = raw.duration

taken_quarter_seconds = np.array([0 for i in range(int(duration * 4))])
for start, stop in a[a[:, 1] - a[:, 0] < 4.1]:
    start = int(start * 4)
    stop = int(stop * 4)
    taken_quarter_seconds[start:stop] += 1

unique_coverage = len(taken_quarter_seconds[taken_quarter_seconds > 0]) / (raw.duration * 4)
print(unique_coverage)  # We use 95% of the original signal